In [ ]:
# Алгоритм нахождения j(tau)  [j-инварианта]

# Вход: D - дискриминант числового поля
#       a, b - коэффициенты квадратичной формы, имеющей дискриминант D.

# Вывод: j-инварианта (Комплексное)

def find_q(tau):
    return exp(2*pi*sqrt(-1)*tau)

def find_etha(q_tau, M):
    s = int(0)
    for m in range(0, M):
        
        s = s + CC(((-1)^m * (q_tau^(m*(3*m-1)/2))+(q_tau^(m*(3*m+1)/2))))
    
    s = 1 + s
    
    s = q_tau^(1/24) * s
    
    return s

def find_delta(etha_tau):
    return (2*pi)^12 * etha_tau^24

def find_j_tau(D, a, b):
    
    d = abs(D)
    
    tau = (-b + sqrt(D)) / (2 * a)
    
    theta = -pi * b / a
    
    p_tau = exp(-pi * sqrt(d) / a)
    
    q_tau = find_q(tau)
    
    q_2tau = find_q(2*tau)
    
    precD = ceil((ln(1)+3.14*sqrt(d))/ln(10)*1) + 10
    
    M = ceil(sqrt(a*2/3*(precD*ln(10)+ln(6))/(pi*sqrt(d))))
    
    etha_tau = find_etha(q_tau, M)
    
    etha_2tau = find_etha(q_2tau, M)
    
    delta_tau = find_delta(etha_tau)
    
    delta_2tau = find_delta(etha_2tau)
    
    f = delta_2tau / delta_tau
    
    j_tau = (256 * f + 1)^3 / (f)
    
    return j_tau.n(precD)
    

In [ ]:
# Алгоритм построения многочлена Гильберта
# ! Алгоритм возвращает не такой класс, как стандартный
# В документвции алгоритм абсолютно такой же используется
# Из-за разных классов возникают трудности с редукцией многочлена,
# так что буду использовать стандартный алгоритм, предоставленный SAGE

# Вход: D - дискриминант числового поля
#       p - модуль
#       a, b - коэффициенты ??
# Вывод: H_D - многочлен Гильберта

def generate_hilbert_polynomial(D, aa, bb):
    
    d = abs(D)

    precD = ceil((ln(1) + pi * sqrt(d)) / ln(10) * 1) + 10
    
    R.<X> = PolynomialRing(CC, 'X')
    
    Priv.<X> = PolynomialRing(RR, 'X')
    
    H_D = 1
    
    H_D in R
    
    b = abs(D) % 2
    
    B = floor(sqrt(abs(D)) / 3)
    
    while b <= B:
        
        t = (b^2 - D) / 4
        a = max(b, 1)
        
        while true:
            if (t % a == 0):
                j_tau = find_j_tau(D, aa, bb)
                
                if (a == b) or (b == 0):
                    H_D = H_D * (X - j_tau)
                else:
                    H_D = H_D * (X^2 - 2 * real(j_tau) * X + j_tau^2)
                    
            a = a + 1
                
            if (a^2 > t):
                break
            
        b = b + 2
        
    return H_D

In [ ]:
# Условия выбора коэффициентов кв. форм

# Вход: D - дискриминант числового поля
#       p - модуль
#       a, b - коэффициенты ??
# Вывод: H_D - многочлен Гильберта

# def find_a_b_c(D):
#     b^2-4ac=-D
#     |b|<=a<=sqrt(D/3)
#     -a<=c
#     gcd(a,b,c)=1
#     if |b|==a or a==c then b >= 0

# Эти значения берутся из таблиц

In [ ]:
# Алгоритм построения кривой и её скручивания по j-инварианту

# Вход: j-инвариант
# Вывод: E - кривая

def build_curve(j, p):
        
    k = j / (1728 - j)
    try:
        k % p
    except ZeroDivisionError:
        k = 1
    a = 3*k % p
    b = 2*k % p
    
    curve = EllipticCurve(GF(p), [3*a, b])
    
    return curve

def build_curve_twist(j, p):
        
    k = j / (1728 - j)
    try:
        k % p
    except ZeroDivisionError:
        k = 1
    a = 3*k % p
    b = 2*k % p
    
    N = [x for x in range(p) if kronecker(x,p) == -1]
    
    c = N[0]
    
    curve = EllipticCurve(GF(p), [3*a*c^2, b*c^3])
    
    return curve

In [ ]:
# Метод КУ на базе многочлена Гильберта

# Вывод: Эллиптическая кривая E

def count_D(prev_D):
    if prev_D == 0:
        return -3
    else:
        while true:
            prev_D = prev_D - 1
            if (prev_D % 4 == 0):
                return prev_D
            if (prev_D % 4 == 1):
                return prev_D

def complex_multiplication_hilbert(t, p = 59):
   
    u = t - p - 1
    
    prev_D = 0
    v = 0
    D = 0
    
    for v in range(1,p+1):
        D = count_D(prev_D)
        if (4*p == u^2 + abs(D) * v^2):
            break
        else:
            prev_D = D
            
#     print u, v, D
    
    H_D =  hilbert_class_polynomial(D)
    
#     print H_D.factor_mod(p)
    
    roots_of_hilbert_polynomial =[]
    
    for root in H_D.factor_mod(p):
        roots_of_hilbert_polynomial.append(root[0].roots())
    
#     print roots_of_hilbert_polynomial
    
    curves = []
    
    for root in roots_of_hilbert_polynomial:
        
        try:
            E = build_curve(root[0][0], p)
            curves.append(E)
        except ArithmeticError:
            E = None
        
        try:
            E_ = build_curve_twist(root[0][0], p)
            curves.append(E_)
        except ArithmeticError:
            E_ = None
    
    return curves

res = complex_multiplication_hilbert(56)

for item in res:
    print item

In [ ]:
def find_z(tau):
    return exp(2*pi*sqrt(-1)*tau)

def find_delta_for_weber(z, M):
    s = int(0)
    for n in range(1, M):
        
        s = s + CC(((-1)^n * (z^(n*(3*n-1)/2))+(z^(n*(3*n+1)/2))))
    
    s = 1 + s
    
    s = z * s^24
    
    return s

def find_F_z(delta, z):
    return (delta / z)^(1/24)

def weber_f_0(theta,z):
    return theta^(-1/24) * find_F_z(-theta, z) / find_F_z(theta^2,z)

def weber_f_1(theta,z):
    return theta^(-1/24) * find_F_z(theta,z) / find_F_z(theta^2,z)

def weber_f_2(theta,z):
    return sqrt(2) * theta^(1/12) * find_F_z(theta^4,z) / find_F_z(theta^2,z)


def generate_weber_polynomial(D):
    a = 1  #из таблиц берем
    b = 1
    c = 2
    
    d = abs(D)
    
    tau = (-b + sqrt(D)) / (2 * a)
    
    z = find_z(tau)
    
    precD = ceil((ln(1)+3.14*sqrt(d))/ln(10)*1) + 10
    
    M = ceil(sqrt(a*2/3*(precD*ln(10)+ln(6))/(pi*sqrt(d))))
    
    delta = find_delta_for_weber(z, M)
    
    theta = z^(-1)
    
    gamma = (weber_f_0(theta,z)^24 + 8) * (weber_f_1(theta,z)^8 - weber_f_2(theta,z)^8) / weber_f_0(theta,z)^8
    
    W_D = 1
    
    if (D % 3 == 0) and (D % 8 != 3):
        if (D % 8 == 7):
            W_D = hilbert_class_polynomial2(4*D, weber_f_0(theta,z)/sqrt(2))
        if (D/4 % 8 == 2) or (D/4 % 8 == 6):
            W_D = hilbert_class_polynomial2(D, weber_f_1(theta,z)/sqrt(2))
        if (D/4 % 8 == 5):
            f_ = weber_f_0(theta,z)
            W_D = hilbert_class_polynomial2(D, f_ ^ 4)
        if (D/4 % 8 == 1):
            f_ = weber_f_0(theta,z)
            W_D = hilbert_class_polynomial2(D, (f_ ^ 2) / sqrt(2))
    if (D % 6 == 3):
        W_D = hilbert_class_polynomial2(D, sqrt(-D) * gamma)
    else:
        W_D = hilbert_class_polynomial(D)
    return W_D
    

In [ ]:
# Метод КУ на базе многочлена Вебера

def complex_multiplication_weber(t, p = 59):
    D = -19
    if (D % 4 != 0 or D % 4 != 3) and D % 8 == 3:
        print "D не соответствует входным условиям!"
        
    W_D = generate_weber_polynomial(D)
    
    u = t - p - 1
    
    prev_D = 0
    v = 0
    D = 0
    
    for v in range(1,p+1):
        D = count_D(prev_D)
        if (4*p == u^2 + abs(D) * v^2):
            break
        else:
            prev_D = D
    
    roots_of_weber_polynomial =[]
    
    for root in W_D.factor_mod(p):
        roots_of_weber_polynomial.append(root[0].roots())
        
    true_roots_weber = []
    
    for root in roots_of_weber_polynomial:
        true_roots_weber.append(root[0])
    
    roots_of_hilbert_polynomial = []
    
    for root_w in true_roots_weber:
        roots_of_hilbert_polynomial.append(power((power(root_w[0],24) - 16), 3) / power(root_w[0],24))
    
    curves = []
    
    for root in roots_of_hilbert_polynomial:
        
        try:
            E = build_curve(root, p)
            curves.append(E)
        except ArithmeticError:
            E = None
        
        try:
            E_ = build_curve_twist(root, p)
            curves.append(E_)
        except ArithmeticError:
            E_ = None
        
    return curves

res = complex_multiplication_weber(56)

for item in res:
    print item


In [ ]:
# Методы КУ на базе многочлена Гильберта с условиями

# Общие входные условия:

# d – положительное целое число, свободное от квадратов
# d (mod 8):      1,2,5         3              6             7
# z_d:           sqrt(-d)  (3+sqrt(-d))/2   3+sqrt(-d)  (-3+sqrt(-d))/2   


# p >= 5
# d – положительное целое число, свободное от квадратов

In [ ]:
# Алгоритм 3.1.  [d ≡ 3,2,1 (mod 4)]   d != 1 и 3

def f_ot_tau(tau):
    dzeta_48 = power(-1,(1/48))
    return dzeta_48 * ((tau+1)/2).eta() / tau.eta()    #Возможно тут надо другую брать ЭТУ
    
def gamma_2(tau):
    return (f_ot_tau(tau) ^ 24 - 16) / f_ot_tau(tau) ^ 8

def gamma_3(tau):
    power((gamma_2(tau) ^ 3 - 1728),(1/2))
    
def is_return_curve(d, U, V):
    if ((d%8 == 7) and ((U-V)%4) == 1) or ((d%8 == 3 and (p%4 == (U+V) and (2*U % 2 == 0)) or ((p%4 == 1 and (2*U % 4 == 3))) or ((p%4 == 3 and (2*V % 4 == 1))))):
        return true
    else:
        return false
        

def algorithm_3_1(d, z_d):
    
    our_j_inv = gamma_3(z_d) * sqrt(-d)
    
    H_D = hilbert_class_polynomial2(-d, our_j_inv)

    roots_of_hilbert_polynomial = []
    
    for root in H_D.factor_mod(p):
        roots_of_hilbert_polynomial.append(root[0].roots())
    
    betha = roots_of_hilbert_polynomial[0]
    
    alpha = -betha * V / U
    
    delta = 1728 + power(alpha,2)
    
    E = EllipticCurve(GF(p), 27 * power(delta, 3), 54 * alpha * power(delta, 4))
    
    E_ = build_curve_twist(betha, p)
    
    if  is_return_curve():
        return E
    else:
        return E_

In [ ]:
def algorithm_3_2(d, z_d):
    
    our_j_inv = gamma_3(z_d) * sqrt(-d)
    
    H_D = hilbert_class_polynomial2(-d, our_j_inv)

    roots_of_hilbert_polynomial = []
    
    for root in H_D.factor_mod(p):
        roots_of_hilbert_polynomial.append(root[0].roots())
    
    betha = roots_of_hilbert_polynomial[0]
    
    alpha = betha * V / U
    
    delta = 1728 + power(alpha,2)
    
    E = EllipticCurve(GF(p), 27 * power(delta, 3), 54 * alpha * power(delta, 4))
    
    E_ = build_curve_twist(betha, p)
    
    if (V % 4 == 1) or (V % 4 == (U-1) % 4):
        return E
    else:
        return E_

In [ ]:
def algorithm_3_3(d, z_d):
    
    our_j_inv = elliptic_j(z_d)
    
    H_D = hilbert_class_polynomial2(-d, our_j_inv)
    
    H_D_ = hilbert_class_polynomial(-d)
    
    delta = sqrt(d) % p
    
    H_D__ = H_D + delta * H_D_
    
    roots_of_hilbert_polynomial = []
    
    for root in H_D__.factor_mod(p):
        roots_of_hilbert_polynomial.append(root[0].roots())
    
    betha = roots_of_hilbert_polynomial[0]
    
    alpha = betha - 1728
    
    etha = power_mod(alpha,((p-1)/4),p)
    
    E = EllipticCurve(GF(p), 27 * power(betha, 3) * alpha, 54 * alpha^2 * power(betha, 4))
    
    E_ = build_curve_twist(betha, p)

    if (V % 2 == 0):
        epsilon = etha % p
        if (U % 4 == epsilon):
            return E
        else:
            return E_
    else:
        i = delta * V / U % p
        if ((etha == i) and (V % 4 == 3)) or ((etha != i) and (V % 4 == 1)):
            return E
        else:
            return E_



In [ ]:
def algorithm_3_4(p):
    
    F = GF(p)
    
    A = 0
    
    if (U % 2 != 0) and (U-1) % 4 == V:
        for a in F:
            if power_mod(a, (p-1)/4, p) == -1:
                A = a
                break
    
        E_a = EllipticCurve(GF(p), A, 0)
        return E_a
    else:
        if (V-1) % 4 == U:
            for a in F:
                if (power_mod(a, (p-1)/4, p) == (U/V)):
                    A = a
                    break
    
            E_a = EllipticCurve(GF(p), A, 0)
            return E_a
            
        else:
            V = -V
            for a in F:
                if power_mod(a, (p-1)/4, p) == (U/V):
                    A = a
                    break
    
            E_a = EllipticCurve(GF(p), A, 0)
            return E_a

In [ ]:
def algorithm_3_5(p):
    
    F = GF(p)
    
    B = 0
    
    if (2*V % 3 == 0) and (2* U % 3 == 2):
        E_b = EllipticCurve(GF(p), 0, 16)
        return E_b
    elif((2*V % 3 == 0) and (2* U % 3 == 2)):
        if (V-1) % 4 == U:
            for b in F:
                if power_mod(a, (p-1)/6, p) == (p-1):
                    B = b
                    break
    
            E_b = EllipticCurve(GF(p), 0, B*16)
            return E_b
    elif((2*V % 3 != 0) and (2* U % 3 == 2)):
        if (2*V) % 3 == 1:
            for b in F:
                if (power_mod(b, (p-1)/6, p) == (2*U/(3*V-U))):
                    B = b
                    break
    
            E_b = EllipticCurve(GF(p), 0, B*16)
            return E_b
            
        else:
            V = -V
            for b in F:
                if power_mod(b, ((p-1)/6), p) == (2*U/(3*V+U)):
                    B = b
                    break
    
            E_b = EllipticCurve(GF(p), 0, B*16)
            return E_b
        

In [ ]:
d = 19

for i in d.factor():
    if i[1] > 1:
        print "ОШИБКА: d должно быть свободно от квадратов!"
        break

d_ = d % 8
z_d = 0

if d_ == 1 or d_ == 2 or d_ == 5:
    z_d = sqrt(-d)
elif (d_ == 3):
    z_d = (3 + sqrt(-d))/2
elif (d_ == 6):
    z_d = (3 + sqrt(-d))
elif (d_ == 7):
    z_d = (-3 + sqrt(-d))/2

print z_d.n()
print real(z_d)

if d % 4 == 3 and d != 3:
    E = algorithm_3_1(d, z_d)

    
if d % 4 == 2:
    E = algorithm_3_2(d, z_d)
    
    
if d % 4 == 1 and d != 1:
    algorithm_3_3(d, z_d)

    
if d == 1:
    algorithm_3_4(p)
    

if d == 3:
    algorithm_3_5(p)
    

In [ ]:
def hilbert_class_polynomial2(D, our_j_invariant=None):
    
    D = Integer(D)
    if D >= 0:
        raise ValueError("D (=%s) must be negative" % D)
    if not (D % 4 in [0, 1]):
        raise ValueError("D (=%s) must be a discriminant" % D)

    # get all primitive reduced quadratic forms, (necessary to exclude
    # imprimitive forms when D is not a fundamental discriminant):

    rqf = BinaryQF_reduced_representatives(D, primitive_only=True)

    h = len(rqf) # class number
    c1 = 3.05682737291380 # log(2*10.63)
    c2 = sum([1/RR(qf[0]) for qf in rqf], RR(0))
    prec =  c2*RR(3.142)*RR(D).abs().sqrt() + h*c1  # bound on log
    prec = prec * 1.45   # bound on log_2 (1/log(2) = 1.44..)
    prec = 10 + prec.ceil()  # allow for rounding error

    # set appropriate precision for further computing

    Dsqrt = D.sqrt(prec=prec)
    R = ComplexField(prec)['t']
    t = R.gen()
    pol = R(1)
    
    if our_j_invariant != None:
        if (our_j_invariant not in rqf):
            pol *= (t - our_j_invariant)
    
#     for qf in rqf:
#         a, b, c = list(qf)
#         tau = (b + Dsqrt) / (a << 1)
#         pol *= (t - elliptic_j(tau))

    coeffs = [cof.real().round() for cof in pol.coefficients(sparse=False)]
    return IntegerRing()['x'](coeffs)


In [ ]:
our_j_invariant = 29
print hilbert_class_polynomial(-19)
print hilbert_class_polynomial2(-19, 10)

In [ ]:
E = complex_multiplication_hilbert(37, 59)

for curve in E:
    print curve

In [ ]:
EE = complex_multiplication_weber(37, 59)

for curve in EE:
    print curve